In [1]:
# Test database module functionality
import sys
sys.path.append('..')  # Add parent directory to path to import our modules

from db.sqlite import (
    get_db,
    initialize_db,
    DatabaseError
)

# 1. Test basic connection
print("Testing database connection...")
try:
    with get_db() as conn:
        print("✓ Connection successful")
except DatabaseError as e:
    print(f"✗ Connection failed: {e}")

# 2. Test simple targeted queries
print("\nTesting specific queries...")
try:
    with get_db() as conn:
        # Test a single PDB
        pdb_query = """
        SELECT COUNT(*) 
        FROM residues 
        WHERE pdb_accession_id = '101m' AND pdb_chain_id = 'A'
        """
        count = conn.execute(pdb_query).fetchone()[0]
        print(f"101m chain A has {count} residues")
        
        # Test a single UniProt
        uniprot_query = """
        SELECT COUNT(DISTINCT pdb_accession_id) 
        FROM residues 
        WHERE uniprot_accession_id = 'P02185'
        """
        count = conn.execute(uniprot_query).fetchone()[0]
        print(f"P02185 appears in {count} PDB structures")

except DatabaseError as e:
    print(f"✗ Query failed: {e}")

# 3. Test our core mapping function
print("\nTesting core mapping query...")
try:
    with get_db() as conn:
        query = """
        SELECT 
            pdb_residue_number,
            pdb_residue_name,
            uniprot_residue_number,
            uniprot_residue_name
        FROM residues 
        WHERE pdb_accession_id = ? 
        AND pdb_chain_id = ? 
        AND pdb_residue_number = ?
        """
        result = conn.execute(query, ('101m', 'A', 1)).fetchone()
        if result:
            print(f"Mapping for 101m/A/1:")
            print(f"  PDB: {result[0]} {result[1]}")
            print(f"  UniProt: {result[2]} {result[3]}")
        else:
            print("No mapping found")
except DatabaseError as e:
    print(f"✗ Mapping query failed: {e}")

Testing database connection...
✓ Connection successful

Testing specific queries...
101m chain A has 154 residues
P02185 appears in 343 PDB structures

Testing core mapping query...
Mapping for 101m/A/1:
  PDB: 1 VAL
  UniProt: 2 V
